In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [3]:

os.chdir('..')
os.getcwd()

'/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - statin/8. RNA-seq/Elizabeth/LW15 analysis/LW15-Functional-Screening'

In [5]:
# read in CRISPR & cDNA library coverage data
CRISPR_path = '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/Functional screening/240223_Natalie_Moe requested genes Query CRISPR 891 for 274.xlsx'
cDNA_path = '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/Functional screening/240318_Natalie_Moe requested genes Query Sigma 226 for 162.xlsx'

CRISPR_lib = pd.read_excel(CRISPR_path, sheet_name=None)
cDNA_lib = pd.read_excel(cDNA_path, sheet_name=None)

CRISPR_query = CRISPR_lib['Moe requested genes Query1']
CRISPR_coverage = CRISPR_lib['Sheet1']

cDNA_query = cDNA_lib['Moe requested genes Query']
cDNA_coverage = cDNA_lib['Gene coverage']

#add coverage data to functional_screening df
# functional_screening['CRISPR_coverage'] = None
# functional_screening['cDNA_coverage'] = None

CRISPR_query.set_index('Ensembl', inplace=True)
cDNA_query.set_index('Ensembl', inplace=True)

CRISPR_coverage.set_index('Gene', inplace=True)
cDNA_coverage.set_index('Moe requested genes_Name', inplace=True)

In [8]:
# Load the HTML content
with open('index.html', 'r') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

# Find the table by ID
table = soup.find('table', {'id': 'functionalScreening'})

In [17]:
# Find index where NaN values are present
nan_indices = CRISPR_coverage.index[CRISPR_coverage.index.isna()]

# Drop these rows from the DataFrame
CRISPR_coverage = CRISPR_coverage.drop(nan_indices)

In [18]:
# Create a new list to store the modified indices
new_indices = [idx[0] + idx[1:].lower() for idx in CRISPR_coverage.index]

# Assign the modified list as the new index of the DataFrame
CRISPR_coverage.index = new_indices

In [29]:
# Create a new list to store the modified indices
new_indices = [idx[0].upper() + idx[1:] for idx in cDNA_coverage.index]

# Assign the modified list as the new index of the DataFrame
cDNA_coverage.index = new_indices

In [21]:
CRISPR_coverage.columns

Index(['384 well key', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Total clones'],
      dtype='object')

In [33]:

# Iterate through each row
for row in table.find_all('tr')[1:]:
    cells = row.find_all('td')
    if cells:
        gene = cells[0].string.strip()  
        
        if gene in cDNA_coverage.index:
            cDNA = cDNA_coverage.loc[gene, 'Total clones']
        else:
            cDNA = 0

        print('*'+gene+'*', cDNA)
        
        cells[3].string = str(cDNA)
        
        if gene in CRISPR_coverage.index:
            CRISPR = int(CRISPR_coverage.loc[gene, 'Total clones'])
        else:
            CRISPR = 0
            
        cells[2].string = str(CRISPR)
        
        if (CRISPR == 0) and (cDNA == 0):
            cells[2].string = 'No coverage'
            cells[3].string = 'No coverage'


# Save the modified HTML
with open('updated_index.html', 'w') as file:
    file.write(str(soup))
                
            


*4931428F04Rik* 0
*Caskin1* 0
*Scarf2* 0
*Vps4a* 1
*Card10* 0
*Thap11* 1
*Nup50* 1
*Rrad* 1
*Eps8l2* 1
*Cited1* 0
*Reep6* 1
*Ccdc92* 1
*Trp53rka* 0
*Gfer* 1
*Shf* 1
*Stac2* 0
*Zfp395* 0
*Sptbn4* 0
*B4galt3* 2
*Sh3bp1* 0
*Rasgrp4* 1
*Spata2l* 2
*Arid3c* 0
*Vill* 2
*Rfng* 0
*Nlgn2* 0
*Syce2* 0
*Prph* 1
*Mkrn1* 0
*Gpr157* 5
*Foxa3* 1
*Zcwpw1* 0
*Prrx2* 0
*Ldhd* 1
*Stard10* 0
*Arsi* 1
*Plvap* 1
*Prss16* 0
*Fam117a* 0
*Trim62* 2
*Eva1c* 0
*Lrrc73* 0
*Tradd* 0
*Dact3* 1
*Ptpn6* 1
*Rtraf* 0
*Gm9747* 0
*Pgf* 0
*Mypop* 1
*Fbxo17* 1
*Hoxc11* 1
*Glo1* 1
*Grik5* 0
*Brsk1* 1
*Slc24a3* 0
*Tmem121* 1
*Plekhg6* 1
*Prss41* 0
*Crnde* 0
*Dpf1* 2
*Crlf1* 0
*Ptpn18* 2
*Abcg4* 0
*Adam11* 0
*Bcar1* 0
*Hoxc5* 0
*Adgrl1* 0
*Dhcr24* 1
*Dpf3* 1
*Batf3* 1
*Cbarp* 0
*Hic2* 0
*Mpzl1* 2
*Tpi1* 0
*Dlgap3* 0
*Gm10157* 0
*Ap1m2* 2
*Samd14* 0
*Map4k1* 1
*2410004I01Rik* 0
*Crabp1* 1
*Fndc4* 1
*A430093F15Rik* 0
*Pcsk4* 0
*Fbxl14* 1
*Tcf15* 0
*Ninl* 0
*Rcor2* 0
*Nckipsd* 1
*Dusp2* 0
*Lrrc23* 0
*Cadm4* 0
*Ecd* 1
*Esrrb* 2
*

In [28]:
cDNA_coverage.loc['Epha4','Total clones']

3